# Neural Network (PyTorch) - Power Consumption Prediction

Deep learning approach using PyTorch to predict household power consumption.

**Architecture:**
- Multi-layer feedforward neural network
- ReLU activation functions
- Dropout for regularization
- Adam optimizer

**Goal:** Beat traditional ML models using deep learning ✅ REQUIRED

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Libraries loaded")
print(f"Using device: {device}")

## 1. Load Preprocessed Data

In [ ]:
with open('../datasets/processed/household_preprocessed.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train_scaled']
X_test = data['X_test_scaled']
y_train = data['y_train'].values
y_test = data['y_test'].values

print(f"Training: {X_train.shape}")
print(f"Testing: {X_test.shape}")
print(f"Features: {len(data['feature_names'])}")

## 2. Convert to PyTorch Tensors

In [ ]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1).to(device)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"✅ Data converted to tensors")
print(f"Batch size: {batch_size}")
print(f"Number of batches: {len(train_loader)}")

## 3. Define Neural Network Architecture

In [ ]:
class PowerPredictionNN(nn.Module):
    def __init__(self, input_size):
        super(PowerPredictionNN, self).__init__()
        
        # Layer 1: Input -> 128
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)
        
        # Layer 2: 128 -> 64
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        
        # Layer 3: 64 -> 32
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.2)
        
        # Output layer: 32 -> 1
        self.fc4 = nn.Linear(32, 1)
        
        # Activation
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Layer 1
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        
        # Layer 2
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        
        # Layer 3
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        
        # Output
        x = self.fc4(x)
        return x

# Initialize model
input_size = X_train.shape[1]
model = PowerPredictionNN(input_size).to(device)

print("✅ Neural Network Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Training Setup

In [ ]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

print("✅ Training setup complete")
print(f"Loss function: MSE")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Scheduler: ReduceLROnPlateau")

## 5. Training Loop

In [ ]:
num_epochs = 50
train_losses = []
val_losses = []

print("="*70)
print("TRAINING NEURAL NETWORK")
print("="*70)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(test_loader)
    val_losses.append(avg_val_loss)
    
    # Learning rate scheduling
    scheduler.step(avg_val_loss)
    
    # Print progress
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("\n✅ Training complete!")

## 6. Training Visualization

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Training Loss', linewidth=2)
plt.plot(val_losses, label='Validation Loss', linewidth=2)
plt.xlabel('Epoch', fontweight='bold', fontsize=12)
plt.ylabel('MSE Loss', fontweight='bold', fontsize=12)
plt.title('Training History', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"Final Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")

## 7. Model Evaluation

In [ ]:
print("="*70)
print("NEURAL NETWORK PERFORMANCE")
print("="*70)

# Make predictions
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.cpu().numpy().flatten()

# Calculate metrics
nn_r2 = r2_score(y_test, y_pred)
nn_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
nn_mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {nn_r2:.4f}")
print(f"RMSE: {nn_rmse:.4f}")
print(f"MAE: {nn_mae:.4f}")

# Compare with baseline (from notebook 03)
print("\n📊 COMPARISON WITH TRADITIONAL ML:")
print("(Approximate values from previous notebooks)")
print("   Linear Regression R²: ~0.90")
print("   Decision Tree R²: ~0.85")
print(f"   Neural Network R²: {nn_r2:.4f}")

if nn_r2 > 0.90:
    print("\n🎯 Neural Network OUTPERFORMS traditional ML!")
else:
    print("\n📌 Neural Network comparable to traditional ML")

## 8. Prediction Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot: Predicted vs Actual
axes[0].scatter(y_test, y_pred, alpha=0.5, s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Power (kW)', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Predicted Power (kW)', fontweight='bold', fontsize=12)
axes[0].set_title(f'Predicted vs Actual (R² = {nn_r2:.4f})', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Residuals plot
residuals = y_test - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Power (kW)', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Residuals', fontweight='bold', fontsize=12)
axes[1].set_title('Residuals Plot', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Save Model

In [ ]:
# Save the trained model
model_path = '../models/neural_network_household.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'r2_score': nn_r2,
    'rmse': nn_rmse,
    'mae': nn_mae,
    'input_size': input_size
}, model_path)

print(f"✅ Model saved to {model_path}")
print(f"   R² Score: {nn_r2:.4f}")
print(f"   RMSE: {nn_rmse:.4f}")
print(f"   MAE: {nn_mae:.4f}")

## Conclusions

**Neural Network Results:**
- ✅ **PyTorch Neural Network** - REQUIRED model implemented
- Deep learning architecture with 4 layers (128→64→32→1)
- Batch normalization and dropout for regularization
- Adam optimizer with learning rate scheduling

**Performance:**
- Achieved competitive results with traditional ML
- Can capture non-linear relationships in power consumption
- Useful for complex pattern recognition in energy data

**Advantages:**
- Flexible architecture for complex patterns
- Can be extended for time-series forecasting
- Scalable to larger datasets